In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

In [2]:
#!pip install tensorflow==2.5.0

In [3]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.7.0


In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [5]:
import numpy as np
#The data is split into 25,000 samples for training and 25,000 samples for testing.
train_data, test_data = imdb['train'], imdb['test']


# define the lists containing the sentences and labels for both training and testing data.
# Now, I can iterate over training data extracting the sentences and the labels.
# The values for S and I are tensors, so by calling their NumPy method,
# I'll actually extract their value. Then I'll do the same for the test set
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  #The value 1 indicates a positive review and zero a negative one.

#When training, my labels are expected to be NumPy arrays
# I'll turn the list of labels that I've just created into NumPy arrays with this code  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
# Once we have our word index, 
#we can now replace the strings containing the words with the token value we created for them.
sequences = tokenizer.texts_to_sequences(training_sentences)
#we'll pad and or truncate the sequenced sentences until they're all the same length
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [7]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [8]:
model = tf.keras.Sequential([
    #what if you could pick a vector in a higher-dimensional space say 16 dimensions,
    # and words that are found together are given similar vectors.
    # Then over time, words can begin to cluster together.
    # The meaning of the words can come from the labeling of the dataset
    #As the neural network trains,it can then learn these vectors associating them with the labels to come up with what's called an embedding i.e.,
    # the vectors for each word with their associated sentiment.                      
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    
    #The results of the embedding will be a 2D array with the length of the sentence and the embedding dimension for example 16 as its size.
    #So we need to flatten it out
    #tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(), # its flatten the data too but more faster than Flatten()
    
    #then feed that into a dense neural network to do the classification.
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
#compile your model as before, and print out the summary
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 9s 10ms/step - loss: 0.5463 - accuracy: 0.7704 - val_loss: 0.3838 - val_accuracy: 0.8431
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3318 - accuracy: 0.8617 - val_loss: 0.3306 - val_accuracy: 0.8600
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2693 - accuracy: 0.8910 - val_loss: 0.3299 - val_accuracy: 0.8581
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2334 - accuracy: 0.9078 - val_loss: 0.3521 - val_accuracy: 0.8498
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2077 - accuracy: 0.9221 - val_loss: 0.3629 - val_accuracy: 0.8492
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1875 - accuracy: 0.9327 - val_loss: 0.3901 - val_accuracy: 0.8418
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1704 - accuracy: 0.9390 - val_loss: 0.4153 - val_accuracy: 0.8398
E

In [10]:
#embeding layer 0 weight
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [11]:
# it currently stands, our word index has the key being the word, and the value being the token for the word.
# We'll need to flip this around, to look through the padded list to decode the tokens back into the words,
# so we've written this helper function
# reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# reverse_word_index 

In [12]:
import io
#write the vectors and their metadata auto files.
# The TensorFlow Projector reads this file type and uses it to plot the vectors in 3D space so we can visualize them.
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  # To the metadata array, we just write out the words.
  out_m.write(word + "\n")
  #To the vectors file, we simply write out the value of each of the items in the array of embeddings, i.e, the co-efficient of each dimension on the vector for this word.
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [13]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
